In [1]:
import json
import torch
import os, sys, random, numpy, pickle
import argparse
import logging
from pprint import pprint
from drop_reader import DropReader
from drop_dataloader import DropBatchGen
from network import QDGATNet
from utils import AverageMeter
from datetime import datetime
from optimizer import AdamW
from utils import create_logger
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler
from drop_dataloader import create_collate_fn
from tqdm.notebook import tqdm

logger = logging.getLogger()
logging.basicConfig(level = logging.INFO)
formatter = logging.Formatter(fmt='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S')
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

In [2]:
bert_model = RobertaModel.from_pretrained('deepset/roberta-base-squad2', output_hidden_states=True)
network = QDGATNet(bert_model,
                hidden_size=bert_model.config.hidden_size,
                dropout_prob=0.1)

Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
network_params_path = os.path.join('./model_15_epoch', 'checkpoint_best.pt')
network.load_state_dict(torch.load(network_params_path))
evaluation = network.eval()

QDGATNet(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps